# Handling Complex Images Convolutional Neural Network

In [4]:
import tensorflow as tf
import os
import zipfile
import urllib
urllib.request.urlretrieve("https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip",
                           filename="/tmp/happy-or-sad.zip")

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

In [17]:
def train_happy_sad_model():
  Desired_acc = 0.999
  class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self,epoch,logs={}):
        if (logs.get('acc')>Desired_acc):
          print('\n Reached 99.9% accuracy so cancelling training')
          self.model.stop_training = True
  callbacks = myCallback()
  model = tf.keras.models.Sequential([
                                      tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(150,150,3)),
                                      tf.keras.layers.MaxPooling2D(2,2),
                                      tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
                                      tf.keras.layers.MaxPooling2D(2,2),
                                      tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                      tf.keras.layers.MaxPooling2D(2,2),
                                      tf.keras.layers.Flatten(),
                                      tf.keras.layers.Dense(512,activation='relu'),
                                      tf.keras.layers.Dense(1,activation='sigmoid')

                  
  ])
  model.summary()
  from tensorflow.keras.optimizers import RMSprop
  model.compile(loss='binary_crossentropy',
                optimizer = RMSprop(lr=0.001),
                metrics=['acc'])
  from tensorflow.keras.preprocessing.image import ImageDataGenerator
  train_datagen = ImageDataGenerator(rescale=1/255)
  train_generator = train_datagen.flow_from_directory(
      '/tmp/h-or-s/',  # This is the source directory for training images
      target_size=(150,150),
      batch_size = 10,
      class_mode = 'binary'
  )
  history = model.fit_generator(
      train_generator,
      steps_per_epoch = 8,
      epochs = 10,
      verbose =1,
      callbacks=[callbacks]
  )
  return history.history['acc'][-1]

In [18]:
train_happy_sad_model()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 18496)            

1.0

##  The reported accuracy is 100% on the training data...